In [1]:
import json
import numpy as np
import random
import nltk

In [2]:
with open("reviews_Books_5.json","r") as data_file:
    raw_data=data_file.readlines()

In [3]:
json_list = []
for i in range(len(raw_data)):
    try:
        json_data = json.loads(raw_data[i])
        json_list.append(json_data)
    except:
        print(raw_data[i],i)

{"reviewerID": "AAXUNK0W2DZG5", "asin": "0060520841", "reviewerName": "Amazon Customer \"leneker\"", "helpful": [5, 10], "reviewText": "1996 Bernard Goldberg wrote an editorial for the Wall Street Journal that said there was an obvious bias on the part of network new shows  for the liberal point of view.  he then illustrated this with an example that he dissected in-depth.  The reaction to this observation was the ruination of her career, and the beginning of his status as a pariah among most newsmen.  This book is used to add more ammo to the controversy.That the journalists who so eagerly pry into other peoples lives and business should be reluctant to be examined is hardly surprising.  Almost no one really wants to have his life or business dissected by Mike Wallace not even Dan Rather.  Some facts in this book are really potent such as the survey results which show how the average journalist and the average American are often vastly at odds.  Other chapters highlight different stor

In [4]:

review_data = []

for i in range(len(json_list)):
    review_dict={}
    review_dict["reviewText"] = json_list[i]["reviewText"]
    review_dict["overall"] = json_list[i]["overall"]
    review_data.append(review_dict)

In [5]:

reviewText = []
for i in range(len(review_data)):
    reviewText.append(review_data[i]["reviewText"])

In [6]:
y_data = []
for i in range(len(review_data)):
    y_data.append(review_data[i]["overall"])

In [7]:

from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import re

In [8]:

def review_to_words(raw_review):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    
   
    cleaned_text_list=[]
    for i in range(len(raw_review)):
        
        review_text = BeautifulSoup(raw_review[i]).get_text()
        letters_only = re.sub("[^a-zA-Z]", " ", review_text)
        words = letters_only.lower().split()
        cleaned_text = " ".join(words)
        cleaned_text_list.append(cleaned_text)

    return cleaned_text_list

In [9]:
pre_data=review_to_words(reviewText)

C:\Users\samsung\Miniconda3\envs\workspace\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\samsung\Miniconda3\envs\workspace\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [10]:
c = list(zip(pre_data, y_data))
random.shuffle(c)

In [11]:
a, b = zip(*c)

In [12]:
text_data=list(a)
overall=np.array(list(b))

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word', sublinear_tf=True,lowercase=True, stop_words='english', ngram_range=(1,2),
                            max_df = 0.2, min_df =2)


tfidf = vectorizer.fit_transform(text_data)

In [14]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(tfidf , overall , test_size=0.33, random_state=100)

In [15]:
from sklearn import linear_model
logreg = linear_model.LogisticRegression(C=0.8, random_state=500, solver='sag', multi_class='multinomial', warm_start=True)

logreg.fit(x_train, y_train)
logreg.predict(x_test)==y_test

# from sklearn import linear_model
# #logreg = linear_model.LogisticRegression()
# logreg = linear_model.LogisticRegression(random_state=42,solver='sag',multi_class='multinomial',warm_start=True)
# logreg.fit(x_train, y_train)

array([ True,  True, False, ..., False,  True, False], dtype=bool)

SVM

In [23]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
result = OneVsRestClassifier(LinearSVC(random_state=0)).fit(x_train, y_train)
result.predict(x_test)
print("SVM :",accuracy_score(result.predict(x_test),y_test))

SVM : 0.624662612598


Logistic

In [16]:
from sklearn.metrics import accuracy_score
accuracy_score(logreg.predict(x_test), y_test)

0.62033754999837398

In [17]:

from sklearn.metrics import precision_score
precision_score(logreg.predict(x_test), y_test,average=None)

array([ 0.15577191,  0.09090909,  0.1894326 ,  0.26325335,  0.94454498])

In [18]:

from sklearn import metrics
for i in range(1,6):
    fpr, tpr, thresholds = metrics.roc_curve(y_test, logreg.predict(x_test), pos_label=i)
#pos_label Label considered as positive and others are considered negative.
    print(i,"점 = ",metrics.auc(fpr, tpr))

1 점 =  0.345477000502
2 점 =  0.2979198345
3 점 =  0.321178304025
4 점 =  0.452163058045
5 점 =  0.674944894276


In [19]:
result=logreg.predict(x_test)

In [20]:
from sklearn.metrics import classification_report
print(classification_report(y_test, result))

             precision    recall  f1-score   support

        1.0       0.65      0.16      0.25      1438
        2.0       0.43      0.09      0.15      1683
        3.0       0.42      0.19      0.26      3331
        4.0       0.45      0.26      0.33      7168
        5.0       0.66      0.94      0.78     17131

avg / total       0.57      0.62      0.56     30751

